In [44]:
import cv2
import mediapipe as mp
import math
import numpy as np
from time import sleep
# from ffpyplayer.player import MediaPlayer
# from mediapipe.python.solutions import hands

In [45]:
def dist(x1,y1,x2,y2): # 비교를 위한 거리 계산 함수
    return math.sqrt( math.pow(x1-x2,2) + math.pow(y1-y2,2))

In [46]:
compareIndex = [[18,4],[6,8],[10,12],[14,16],[18,20]] # 마디 index 번호
hand_open = [False,False,False,False,False] # False로 초기화
gesture = [
    [False,True,True,False,False,"Yeah"],
    [True,False,False,False,False,"Good"],
]

In [47]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mpHands = mp.solutions.hands
my_hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

In [48]:
#control frame rate
cap = cv2.VideoCapture('../sample_dance/videoplayback.mp4') # jupyter 
# cap = cv2.VideoCapture('./sample_dance/videoplayback.mp4') # vs

#cap = cv2.VideoCapture('C:/Users/yoond/바탕 화면/00CAPSTONE/Capstone-Design/DAIN/sample_dance/videoplayback.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

frame_counter = 0
frameTime = int((1/fps)*1000)  #time of each frame (ms단위, 몇ms당 1frame으로 할지 설정)
print(fps)
extract_time_by_per_frame = 1 #몇프레임 당 한번 측정할지 조절 가능


30.0


In [49]:
# ###start step2 --> 영상 추출
# #extract_file = open('DAIN/extractsamples/result.txt','w')
# cap = cv2.VideoCapture('../sample_dance/videoplayback.mp4') # jupyter 
# # cap = cv2.VideoCapture('./sample_dance/videoplayback.mp4') # vs

# #cap = cv2.VideoCapture('C:/Users/yoond/바탕 화면/00CAPSTONE/Capstone-Design/DAIN/sample_dance/videoplayback.mp4')
# fps = cap.get(cv2.CAP_PROP_FPS)



result = np.array([]) # 추출된 영상의 전체 넘파이 배열
bone_index = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28] # 필요한 관절 번호


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame_counter += 1 #increase frame counter

        #Recolor image to RGB
        if ret:
            if frame_counter % extract_time_by_per_frame == 0:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                #Detection
                results = pose.process(image)

                #Recolor image RGB to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                #Extract landmarks
                try: #success to extract landmarks
                    landmarks = results.pose_landmarks.landmark
                    
                    ret = []
                    
                    # # 11 12 13 14 15 16 23 24 25 26 27 28
                    for i in bone_index: # 필요한 부분의 관절의 정보만
                        temp = np.array([landmarks[i].x,landmarks[i].y,landmarks[i].z], float)
                        ret = np.append(ret,temp,axis = 0)
                        
                        
#                     for i in range(len(landmarks)):
#                         temp = np.array([landmarks[i].x,landmarks[i].y,landmarks[i].z], float)
#                         ret = np.append(ret,temp,axis = 0)
                    
                    #print(ret)
                    result = np.append(result,ret,axis=0)
                    #print("extracting\n")
                except:
                    pass
        
        else: #no next frame (end point of video)
            break
            #Show video on screen
        cv2.imshow('dance_file',frame)
        
        if cv2.waitKey(frameTime) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    #extract_file.close()





In [50]:
print('*****extract success*****')
z = result.shape[0]/(len(bone_index)*3)
result = np.reshape(result,(int(z),len(bone_index),3)) # frame 수,  tracking좌표 수, xyz 
print(result[0])
print(result.shape)

*****extract success*****
[[ 0.56878728  0.36568233 -0.15027806]
 [ 0.49939519  0.37074822 -0.1233276 ]
 [ 0.58659947  0.45521194 -0.125323  ]
 [ 0.49071422  0.46475789 -0.09052768]
 [ 0.57153028  0.53883195 -0.20202181]
 [ 0.50347084  0.54858035 -0.17785837]
 [ 0.55581081  0.54644543 -0.01378596]
 [ 0.51830608  0.54936028  0.01372805]
 [ 0.55500901  0.69884503 -0.0302585 ]
 [ 0.51074547  0.70174855  0.00140939]
 [ 0.55349058  0.81681919  0.13963871]
 [ 0.52232307  0.80851245  0.1724093 ]]
(299, 12, 3)


# body index
body_index = [
    
    [11,13,15], # left arm
    [12,14,16], # right arm
    [23,25,27], # left leg
    [24,26,28], # right leg
    
    [14,12,24],
    [13,11,23],
    [23,24,26],
    [24,23,25],
    
]
l1 = result[0][11]
l2 = result[0][13]
l3 = result[0][15]


ll1 = l1-l2
ll2 = l3-l2

innerAB = np.dot(ll1,ll2)
AB = np.linalg.norm(ll1) * np.linalg.norm(ll2)
angle = np.arccos(innerAB/AB)
print(angle) # radian
print(angle/np.pi*180)

In [51]:
body_index = [
    [0,2,4], # left arm
    [1,3,4], # right arm
    [6,8,10], # left leg
    [7,9,11], # right leg

    [3,1,7],
    [2,0,6],
    [6,7,9],
    [7,6,8],
]

In [55]:
user_video_result = result # 녹화된 비디오
total_point = 0 # 총 포인트

for n in range(len(result)-1, -1,-1):
    
    for index in body_index:
        l1 = result[n][index[0]]
        l2 = result[n][index[1]]
        l3 = result[n][index[2]]

        ll1 = l1-l2
        ll2 = l3-l2
        
        innerREAL = np.dot(ll1,ll2)
        REAL = np.linalg.norm(ll1) * np.linalg.norm(ll2)
        angleREAL = np.arccos(innerREAL/REAL)/np.pi*180
        #print(angleREAL/np.pi*180)


        u1 = user_video_result[n][index[0]]
        u2 = user_video_result[n][index[1]]
        u3 = user_video_result[n][index[2]]

        uu1 = u1-u2
        uu2 = u3-u2
        
        innerUSER = np.dot(uu1,uu2)
        USER = np.linalg.norm(uu1) * np.linalg.norm(uu2)
        angleUSER = np.arccos(innerUSER/USER)/np.pi*180
        #print(angleUSER)


        if angleUSER <= angleREAL + 10.0 and angleUSER >= angleREAL-10.0:
            total_point+=1
            
print(total_point)
print((total_point / (8*len(result))) * 100) # 8 : 비교 각도의 개수 len(result) : frame 개수

2392
100.0


for num, r in enumerate(result):
    
    for index in body_index:
        l1 = r[index[0]]
        l2 = r[index[1]]
        l3 = r[index[2]]
        
        ll1 = l1-l2
        ll2 = l3-l2
        
        innerAB = np.dot(ll1,ll2)
        AB = np.linalg.norm(ll1) * np.linalg.norm(ll2)
        angle = np.arccos(innerAB/AB)
        print(angle/np.pi*180)
    
    print('\n')
    
    